## OCR

In [11]:
import easyocr

In [15]:
reader = easyocr.Reader(['en'], detector='DB', recognizer = 'Transformer') 

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
result = reader.readtext('image-1.jpg')


In [4]:
result

[([[914, 806], [1304, 806], [1304, 1026], [914, 1026]],
  'blodk',
  0.581792572052716),
 ([[1386, 758], [1850, 758], [1850, 974], [1386, 974]],
  'Fram 2',
  0.3073062988438498),
 ([[538, 830], [830, 830], [830, 1078], [538, 1078]],
  'Fon',
  0.8022684795726847)]

In [5]:
for text in result:
    print(text[1])

blodk
Fram 2
Fon


In [16]:
# a function that takes an image and returns the text in the image
def read_text(image_path):
    result = reader.readtext(image_path)
    text = ''
    for i in result:
        text += i[1] + ' '
    return text

In [17]:
image_path = 'image-1.jpg'
print(read_text(image_path))

blodk Fram 2 Fon 


In [18]:
image_path = 'image-2.jpg'
print(read_text(image_path))

Szndaltuted Fan 


In [19]:
image_path = 'image-3.jpg'
print(read_text(image_path))

blue ParaSel Qapec 


In [20]:
image_path = 'image-4.jpg'
print(read_text(image_path))

Cartoon Qena ( Pener5 Slr 


## Product Image Webscrapping

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/CNN_Model_Train_Data.csv')
data

,StockCode
0,85123A
1,ö85123A^
2,ö22423^
3,22423
4,ö85099B^
5,85099B
6,ö47566^
7,ö20725^
8,47566
9,20725


In [3]:
# cleaning the data
data['StockCode'] = data['StockCode'].str.replace(r'\D', '', regex=True)
data = data.astype(float)
data

,StockCode
0,85123.0
1,85123.0
2,22423.0
3,22423.0
4,85099.0
5,85099.0
6,47566.0
7,20725.0
8,47566.0
9,20725.0


In [4]:
stock_data = pd.read_csv('../data/stock_data.csv')
stock_data

,StockCode,Description
0,85123.0,POSTAGE
1,71053.0,Dotcomgiftshop Gift Voucher 1000
2,84406.0,Dotcomgiftshop Gift Voucher 2000
3,84029.0,Dotcomgiftshop Gift Voucher 3000
4,22752.0,Dotcomgiftshop Gift Voucher 4000
...,...,...
3349,23562.0,GREEN ENAMELGLASS HAIR COMB
3350,23561.0,CLEAR ACRYLIC FACETED BANGLE
3351,23609.0,DIAMOND LAS VEGAS NECKLACE 45CM
3352,23617.0,JET BLACK LAS VEGAS BRACELET ROUND


In [5]:
# getting the description of the stock code
filtered_data = stock_data[stock_data['StockCode'].isin(data['StockCode'])].reset_index(drop=True)
filtered_data

,StockCode,Description
0,85123.0,POSTAGE
1,85099.0,LETTER SHAPE PENCIL SHARPENER
2,20725.0,KITTY PENCIL ERASERS
3,22423.0,TOAST ITS FAIRY FLOWER
4,47566.0,HEART FILIGREE DOVE LARGE


In [6]:
filtered_data.nunique()

StockCode      5
Description    5
dtype: int64

In [7]:
filtered_data['Description'].unique()

array(['POSTAGE', 'LETTER SHAPE PENCIL SHARPENER', 'KITTY PENCIL ERASERS',
       'TOAST ITS  FAIRY FLOWER', 'HEART FILIGREE DOVE LARGE'],
      dtype=object)

## Product Image Link Scrapping

## Amazon -- Scrapping from amazon

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ChromeOptions

In [35]:
# Renaming these objects to make them more understandable
object = ['CUSHION COVER', 'Pencil Sharpener', 'Pencil Eraser', 'WASHBAG', 'BASKET', 'COLOURING PENCILS', 'ANIMAL STICKERS', 'RED PURSE']
len(object)

8

In [33]:
# options = ChromeOptions()
# options.add_argument("--headless=new")
# driver = webdriver.Chrome(options=options)
# url = "https://www.amazon.com/s?k=POCKET+BOOK&ref=nb_sb_noss_2"
# driver.get(url)
# results = []
# content = driver.page_source
# soup = BeautifulSoup(content, "html.parser")
# driver.quit()

# def parse_image_urls(soup, classes, location, source):
#     for a in soup.findAll(attrs={"class": classes}):
#         name = a.find(location)
#         if name not in results:
#             results.append(name.get(source))

# parse_image_urls(soup, "a-section aok-relative s-image-square-aspect", "img", "src")

In [10]:
import time

def parse_image_urls(soup, classes, location, source):
    for a in soup.findAll(attrs={"class": classes}):
        name = a.find(location)
        if name not in results:
            results.append(name.get(source))

def get_img_links(homepage, recurrent_links, no_of_pages, class_name, location, source):
    options = ChromeOptions()
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=options)
    url = homepage
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")
    parse_image_urls(soup, class_name, location, source)
    print(f"Page 1 done")
    time.sleep(2)
    # loop through the pages
    for i in range(2, no_of_pages + 1):
        options = ChromeOptions()
        options.add_argument("--headless=new")
        driver = webdriver.Chrome(options=options)
        url = recurrent_links + str(i)
        driver.get(url)
        content = driver.page_source
        soup = BeautifulSoup(content, "html.parser")
        parse_image_urls(soup, class_name, location, source)
        print(f"Finished scraping page {url}")
        print(len(results))
        time.sleep(2)
    driver.quit()
    return results

In [ ]:
for i in range(len(object)):
    object_ = object[i]
    print(object_)
    obj_name= object_.replace(" ", "+")
    homepage = 'https://www.amazon.com/s?k=%27'+str(obj_name)+'%27&ref=nb_sb_noss_2'
    recurrent_links = 'https://www.amazon.com/s?k=%'+str(obj_name)+'%27&page=2&ref=sr_pg_'
    no_of_pages = 9
    class_name = "a-section aok-relative s-image-square-aspect"
    location = "img"
    source = "src"
    results = []
    object_links = get_img_links(homepage, recurrent_links, no_of_pages, class_name, location, source)
    # creating a dataframe
    image_urls = pd.DataFrame()
    image_urls['image_url'] = object_links
    image_urls.to_csv(str(object_)+'.csv', index=False)